## <center>**🚀 Data Cleaning && Feature Engineering**

#### **Import necessary libraries**

In [1]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

# Set visualization style
plt.style.use('ggplot')

## **📌 Step 1: Load Data**

- ✅ This fetches historical stock data from Yahoo Finance.
- ✅ Columns include: Open, High, Low, Close, Adj Close, and Volume.

In [2]:
# Define the stock symbol (e.g., Apple - AAPL)
stock_symbol = "AAPL"

# Fetch historical stock data (last 1 year)
stock_data = yf.download(stock_symbol, period="5y", interval="1d")

# Display the first few rows
stock_data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-04-03,58.585148,59.626239,57.993010,58.922472,129880000
2020-04-06,63.695953,63.851264,60.519286,60.888156,201820400
2020-04-07,62.958221,65.935898,62.853871,65.717481,202887200
2020-04-08,64.569618,64.885098,63.395052,63.761493,168895200
2020-04-09,65.035553,65.540330,64.237144,65.207860,161834800


#### **📌 Step 2: Data Cleaning**

In [3]:
stock_data.columns = [col[0] for col in stock_data.columns]

In [4]:
stock_data.head(2)

,Close,High,Low,Open,Volume
Date,,,,,
2020-04-03,58.585148,59.626239,57.993010,58.922472,129880000
2020-04-06,63.695953,63.851264,60.519286,60.888156,201820400


In [5]:
stock_data.shape

(1256, 5)

- 1️⃣ Check for missing values

In [6]:
missing_values = stock_data.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 Close     0
High      0
Low       0
Open      0
Volume    0
dtype: int64


- 2️⃣ Handle missing values (Forward Fill method)

In [7]:
stock_data.fillna(method='ffill', inplace=True)

/tmp/ipykernel_7712/2907547834.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data.fillna(method='ffill', inplace=True)


- 3️⃣ Check for duplicates

In [8]:
duplicates = stock_data.duplicated().sum()
print("\nDuplicate Rows:", duplicates)


Duplicate Rows: 0


- 4️⃣ Convert Index to DateTime Format

In [9]:
stock_data.index = pd.to_datetime(stock_data.index)

- 5️⃣ Remove outliers using IQR method

In [10]:
Q1 = stock_data['Close'].quantile(0.25)
Q3 = stock_data['Close'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [11]:
stock_data[(stock_data['Close'] <= lower_bound) & (stock_data['Close'] >= upper_bound)].shape

(0, 5)

In [12]:
stock_data = stock_data[(stock_data['Close'] >= lower_bound) & (stock_data['Close'] <= upper_bound)]

- 6️⃣ Check data types

In [13]:
print("\nData Types:\n", stock_data.dtypes)


Data Types:
 Close     float64
High      float64
Low       float64
Open      float64
Volume      int64
dtype: object


- Display cleaned data

In [14]:
stock_data.head()

,Close,High,Low,Open,Volume
Date,,,,,
2020-04-03,58.585148,59.626239,57.993010,58.922472,129880000
2020-04-06,63.695953,63.851264,60.519286,60.888156,201820400
2020-04-07,62.958221,65.935898,62.853871,65.717481,202887200
2020-04-08,64.569618,64.885098,63.395052,63.761493,168895200
2020-04-09,65.035553,65.540330,64.237144,65.207860,161834800


In [15]:
stock_data.shape

(1256, 5)

**📌 What we have done?**
- ✅ Handles missing values using Forward Fill (previous value replaces NaN).
- ✅ Removes duplicate rows if found.
- ✅ Ensures DateTime indexing for time series analysis.
- ✅ Removes outliers using Interquartile Range (IQR) method.

## **📌 Step 3: Feature Engineering**

- 1️⃣ Calculate Daily Returns (Percentage Change)

In [16]:
stock_data['Daily_Return'] = stock_data['Close'].pct_change()

- 2️⃣ Calculate Logarithmic Returns

In [17]:
stock_data['Log_Return'] = np.log(stock_data['Close'] / stock_data['Close'].shift(1))

- 3️⃣ Compute Cumulative Return

In [18]:
stock_data['Cumulative_Return'] = (1 + stock_data['Daily_Return']).cumprod()

- 4️⃣ Compute Moving Averages (Momentum Indicators)

In [19]:
stock_data['MA_10'] = stock_data['Close'].rolling(window=10).mean()
stock_data['MA_20'] = stock_data['Close'].rolling(window=20).mean()
stock_data['MA_50'] = stock_data['Close'].rolling(window=50).mean()

- 5️⃣ Compute Rolling Volatility (Risk Indicator)

In [20]:
stock_data['Rolling_Volatility'] = stock_data['Daily_Return'].rolling(window=20).std()

- 6️⃣ Compute Bollinger Bands

In [21]:
stock_data['Bollinger_Upper'] = stock_data['MA_20'] + (stock_data['Rolling_Volatility'] * 2)
stock_data['Bollinger_Lower'] = stock_data['MA_20'] - (stock_data['Rolling_Volatility'] * 2)

In [22]:
# Drop NaN values after feature engineering
stock_data.dropna(inplace=True)

# Display cleaned, feature-engineered dataset
stock_data.head()

,Close,High,Low,Open,Volume,Daily_Return,Log_Return,Cumulative_Return,MA_10,MA_20,MA_50,Rolling_Volatility,Bollinger_Upper,Bollinger_Lower
Date,,,,,,,,,,,,,,
2020-06-15,83.461800,84.116374,80.928671,81.091710,138808800,0.012367,0.012291,1.424624,81.523868,79.403079,73.428827,0.017502,79.438083,79.368076
2020-06-16,85.673706,85.946249,83.882757,85.522839,165428800,0.026502,0.026157,1.462379,82.223213,79.854710,73.970598,0.017672,79.890055,79.819366
2020-06-17,85.554489,86.481599,85.432821,86.420765,114406400,-0.001392,-0.001392,1.460344,82.867325,80.322523,74.407769,0.017549,80.357620,80.287425
2020-06-18,85.588554,86.007093,84.977779,85.510685,96820400,0.000398,0.000398,1.460926,83.582976,80.717944,74.860376,0.017293,80.752529,80.683358
2020-06-19,85.099449,86.763866,83.987402,86.296665,264476000,-0.005715,-0.005731,1.452577,84.026334,81.117865,75.270972,0.017231,81.152328,81.083403


In [23]:
# Save cleaned & feature-engineered dataset
stock_data.to_csv(f"{os.path.join("../data/")}AAPL_processed.csv", index=True)